In [164]:
import csv
import json
import os
import operator
from itertools import compress 
import numpy as np
import copy 

# loading human generated questions, answers, paragraphs from squad

In [126]:
root_dir = '/Users/tonytu/desktop/old_stuff/berkeley/fall2020/research'
def load(dataset):
    filepath = os.path.join(root_dir, dataset)
    with open(filepath) as file:
        data = json.load(file)
    return data['data']
    #print data by paragraphs
    print('succesfully loaded {} articles from squad dataset'.format(len(data['data'])))
    print(filepath)
#     return data['data']

def parse(data):
    #extract all paragraphs from json 
    titles = []
    paragraphs = []
    questions = []
    final_questions = []
    answers = []
    for i in range(len(data)):
        title = data[i]['title']
        titles.append(title)
        for j in range(len(data[i]['paragraphs'])):
            if j == 0:
                paragraphs.append([])
                questions.append([])
                answers.append([])
#             print(data[i]['paragraphs'][j]['qas'])
            questions[-1].append(data[i]['paragraphs'][j]['qas'])
            answers[-1].append(data[i]['paragraphs'][j]['qas'])
            paragraphs[-1].append(data[i]['paragraphs'][j]['context'])
    for i in range(len(questions)):
        for j in range(len(questions[i])):
            temp = questions[i][j]
#             print(temp)
            questions[i][j] = []
            answers[i][j] = []
            for k in range(len(temp)):
                questions[i][j].append(temp[k]['question']) 
                if len(temp[k]['answers']) != 0:
                    answers[i][j].append(temp[k]['answers'][0])
                else:
                    answers[i][j].append('')
    return titles, paragraphs, questions, answers

squad_2_dev = load('dev-v2.0.json')
squad_2_train = load('train-v2.0.json')
squad_1_dev = load('dev-v1.1.json')
squad_1_train = load('train-v1.1.json')
# squad_2_dev = json.load()
# squad_2_train
# squad_1_dev
# squad_1_train
titles, paragraphs, questions, answers = parse(squad_1_train)
assert len(paragraphs) == len(questions) == len(answers)

# Heuristics (binary classification) 

In [100]:
def longest_common_substring(s1, s2):
    m = [[0] * (1 + len(s2)) for i in range(1 + len(s1))]
    longest, x_longest = 0, 0
    for x in range(1, 2):
        for y in range(1, 1 + len(s2)):
            if s1[x - 1] == s2[y - 1]:
                m[x][y] = m[x - 1][y - 1] + 1
                if m[x][y] > longest:
                    longest = m[x][y]
                    x_longest = x
            else:
                m[x][y] = 0
    return s1[x_longest - longest: x_longest]

def longest_common_opener(s1, s2):
    common_string = ''
    s1_split = s1.split(" ")
    s2_split = s2.split(" ")
    for i in range(len(s1_split)):
        if s1_split[i] == s2_split[i]:
            common_string += s1_split[i]
            common_string += " "
            continue
        else:
            break
    return common_string

def longest_common_sentence(s1, s2):
    s1_words = s1.split(' ')
    s2_words = s2.split(' ')  
    return ' '.join(longest_common_substring(s1_words, s2_words))

def extract_sentences(file_path):
#     with open('textbooktony.csv', 'r') as file:
    with open(file_path, 'r') as file:
        texts = []
        causes = []
        effects = []
        cause_questions = []
        effect_questions = []
        reader = csv.reader(file)
        for row in reader:
            text = row[2]
            cause = row[3]
            effect = row[4]
            cause_question = row[5]
            effect_question = row[6]
            texts.append(text)
            causes.append(cause)
            effects.append(effect)
            cause_questions.append(cause_question)
            effect_questions.append(effect_question)
    return texts, causes, effects, cause_questions, effect_questions

s1 = 'this is a foo bar sentence .'
s2 = 'this is what a kappa foo bar black sheep ?'
# common_sentence = longest_common_sentence(s1, s2)
common_sentence = longest_common_opener(s1, s2)
print (common_sentence)

this is 


In [101]:
def CountFrequency(my_list): 
    # Creating an empty dictionary  
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq
    for key, value in freq.items(): 
        print ((key, value)) 

## Most common sentence openers from squad dataset (generated from prophetnet)

In [29]:
texts, causes, effects, cause_questions, effect_questions = extract_sentences('squad_ce.csv')
cause_questions.extend(effect_questions)

In [32]:
#compare every distinct pair of question 
common_substrings = []
for i in range(len(cause_questions)-1):
    for j in range(i+1, len(cause_questions)):
        common_sentence = longest_common_opener(cause_questions[i], cause_questions[j])
        common_substrings.append(common_sentence)

In [33]:
cause_counts = CountFrequency(common_substrings)
cause_frequencies = sorted(cause_counts.items(),key=operator.itemgetter(1),reverse=True)
cause_frequencies

[('', 1706450),
 ('what ', 541346),
 ('why ', 68695),
 ('how ', 39972),
 ('what is ', 24049),
 ('what is the ', 12533),
 ('what did ', 9081),
 ('why did ', 5501),
 ('what was the ', 3546),
 ('what happens ', 3340),
 ('why is ', 3260),
 ('what was ', 3033),
 ('when ', 2592),
 ('what does ', 2196),
 ('what did the ', 2099),
 ('who ', 1795),
 ('how did ', 1555),
 ('what are ', 1257),
 ('why did the ', 1225),
 ('how can ', 948),
 ('how do ', 800),
 ('why are ', 763),
 ('what happens if ', 759),
 ('why was ', 734),
 ('what do ', 680),
 ('how is ', 466),
 ('where ', 462),
 ('why do ', 418),
 ('what does the ', 401),
 ('how does ', 400),
 ('what can ', 373),
 ('why does ', 359),
 ('what happens when ', 346),
 ('what is a ', 345),
 ('why is the ', 320),
 ('what has ', 311),
 ('what are the ', 298),
 ('are ', 292),
 ('why were ', 269),
 ('is ', 267),
 ('why was the ', 251),
 ('what would ', 248),
 ('when did ', 232),
 ('how many ', 204),
 ('what happened ', 192),
 ('what causes ', 163),
 ('how 

## Grouping

In [39]:
def classifier(frequencies):
    #binary classifier used to classify the questions as causal
    #or non-causal, if the question starts with why, or any variation
    #of what happens, or contain the token "effect" or "cause", it will
    #be classified as a causal question, otherwise non-causal.
    openers = []
    for i in range(len(frequencies)):
        openers.append(frequencies[i][0])
    cutoff = int(len(frequencies) / 5)
    most_common_openers = openers[:cutoff]
    
    #classification
    causal_openers = []
    non_causal_openers = []
    for opener in most_common_openers:
        if opener.lower().startswith('why'):
            causal_openers.append(opener)
            #if opener does not start with why 
        elif opener.lower().startswith('what happens') or \
            opener.lower().startswith('what would happen') or \
            opener.lower().startswith('what will happen') or \
            opener.lower().startswith('what happened'):
            causal_openers.append(opener)
        elif opener.lower().find('effect') != -1 or opener.lower().find('cause') != -1:
                causal_openers.append(opener)
        else:
            non_causal_openers.append(opener)
    return causal_openers, non_causal_openers

causal_openers, non_causal_openers = classifier(cause_frequencies)

# Using the heuristic to filter the human-generated causal questions from the non-causal in squad dataset

In [226]:
import json
import os
root_dir = '/Users/tonytu/desktop/old_stuff/berkeley/fall2020/research'
def load(dataset):
    filepath = os.path.join(root_dir, dataset)
    with open(filepath) as file:
        data = json.load(file)
    return data['data']
    #print data by paragraphs
    print('succesfully loaded {} articles from squad dataset'.format(len(data['data'])))
    print(filepath)
#     return data['data']

def parse(data):
    #extract all paragraphs from json 
    titles = []
    paragraphs = []
    questions = []
    final_questions = []
    answers = []
    for i in range(len(data)):
        title = data[i]['title']
        titles.append(title)
        for j in range(len(data[i]['paragraphs'])):
            if j == 0:
                paragraphs.append([])
                questions.append([])
                answers.append([])
#             print(data[i]['paragraphs'][j]['qas'])
            questions[-1].append(data[i]['paragraphs'][j]['qas'])
            answers[-1].append(data[i]['paragraphs'][j]['qas'])
            paragraphs[-1].append(data[i]['paragraphs'][j]['context'])
    #padding questions with no answers with empty strings
    for i in range(len(questions)):
        for j in range(len(questions[i])):
            temp = questions[i][j]
#             print(temp)
            questions[i][j] = []
            answers[i][j] = []
            for k in range(len(temp)):
                questions[i][j].append(temp[k]['question']) 
                if len(temp[k]['answers']) != 0:
                    answers[i][j].append(temp[k]['answers'][0])
                else:
                    answers[i][j].append('')
    #multiplying the paragraph array with the number of questions
    #comment out if no need to duplicate
    for i in range(len(questions)):
        for j in range(len(questions[i])):
            temp = paragraphs[i][j]
            paragraphs[i][j] = [temp] * len(questions[i][j])
    return titles, paragraphs, questions, answers
titles, paragraphs, questions, answers = parse(squad_1_train)

In [227]:
#filtering out the non-causal sentences
from collections.abc import Sequence

def get_shape(lst, shape=()):
    """
    returns the shape of nested lists similarly to numpy's shape.

    :param lst: the nested list
    :param shape: the shape up to the current recursion depth
    :return: the shape including the current depth
            (finally this will be the full depth)
    """
    if not isinstance(lst, Sequence):
        # base case
        return shape
    # peek ahead and assure all lists in the next depth
    # have the same length
    if isinstance(lst[0], Sequence):
        l = len(lst[0])
        if not all(len(item) == l for item in lst):
            msg = 'not all lists have the same length'
            raise ValueError(msg)
    shape += (len(lst), )
    # recurse
    shape = get_shape(lst[0], shape)
    return shape

def is_causal(question, causal_openers):
    return any(question.startswith(opener) for opener in causal_openers)

#get indices of questions which are causal
def get_indices(paragraphs, questions, causal_openers):
    indices = copy.deepcopy(paragraphs)
    for i in range(len(paragraphs)):
        for j in range(len(paragraphs[i])):
            for k in range(len(questions[i][j])):
                if is_causal(questions[i][j][k], causal_openers):
#                     print(questions[i][j][k])
                    indices[i][j][k] = True
                else:
                    indices[i][j][k] = False
    return indices

def flatten(t):
    #flattening list
    flat_list = [item for sublist in t for item in sublist]
    return flat_list
    
indices = get_indices(paragraphs, questions, causal_openers)

In [217]:
# test_list = [["i", 'a'], ["a"], ["I"]]
# bool_list = [[True, True], [False], [True]]
# res = list(compress(test_list, bool_list)) 
# causal_questions = list(compress(questions, indices))
# causal_paragraphs = list(compress(paragraphs, indices))
# causal_answers = list(compress(answers, indices))

In [228]:
paragraphs = flatten(flatten(paragraphs))
questions = flatten(flatten(questions))
answers = flatten(flatten(answers))
indices = flatten(flatten(indices))
# indices = flatten(flatten(indices))
# causal_questions = list(compress(questions, indices))
# causal_paragraphs = list(compress(paragraphs, indices))
# causal_answers = list(compress(answers, indices))

In [231]:
causal_questions = list(compress(questions, indices))
causal_paragraphs = list(compress(paragraphs, indices))
causal_answers = list(compress(answers, indices))

## output csv files with causal paragraphs, causal questions, and causal answers